In [117]:
from matplotlib import pyplot as plt
import numpy as np
import json
import math
import csv
import os

In [118]:
def denormalization(arr, angle, translation):

    theta = (angle)/180*math.pi
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    
    #rotate
    arr = np.array(R.dot(arr.transpose())).transpose()
    #translate
    arr += translation
#     arr[...,0] += (AGENT[19,0] - AGENT[0,0])
#     arr[...,1] += (AGENT[19,1] - AGENT[0,1])
    return arr

In [119]:
def prediction(file):
    json_dict = {}

    with open(file, 'r') as json_data:
        json_dict = json.load(json_data)

    preds = np.array(json_dict['preds'][0])[...,:2]
    preds = np.array([denormalization(p, -json_dict['rotation'], -np.array(json_dict['translation'])) for p in preds])
    seq_id = int(json_dict['csv_file'].split('.')[0])
    return preds, seq_id

In [120]:
from tqdm.notebook import tqdm
from argoverse.evaluation.competition_util import generate_forecasting_h5


In [121]:
from glob import glob

file_names = glob("ResultsJson/test/*.json")
output_path = "competition_files/" + "test/"



In [122]:

output_all = {}
for file in tqdm(file_names):
    p, seq_id = prediction(file)
    output_all[seq_id] = p 
generate_forecasting_h5(output_all, output_path) #this might take awhile


  0%|          | 0/78143 [00:00<?, ?it/s]

78143/78143

In [83]:
def prediction(trajectory,num_points=30,avg_points=1):
    #a simple prediction function that predict straight line with constant velocity
    velocity_x_mps = []
    velocity_y_mps = []
    for i in range(1,avg_points+1,1):
        velocity_x_mps.append(trajectory[-i,0] - trajectory[-(1+i),0])
        velocity_y_mps.append(trajectory[-i,1] - trajectory[-(1+i),1])
        
    velocity_x_mps = np.mean(velocity_x_mps)
    velocity_y_mps = np.mean(velocity_y_mps)
    
    current_traj = trajectory[-1]
    results = np.zeros((len(trajectory)+num_points,2))
    
    results[0:len(trajectory)] = trajectory
    
    for i in range(num_points):
        results[len(trajectory)+i] = np.array([current_traj[0]+velocity_x_mps,current_traj[1]+velocity_y_mps])
        current_traj = results[len(trajectory)+i]
    return results


In [88]:
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader

##set root_dir to the correct path to your dataset folder
root_dir = '../DataSet/ArgoverseDataset/test_obs/data'

afl = ArgoverseForecastingLoader(root_dir)

print('Total number of sequences:',len(afl))


Total number of sequences: 78143


In [89]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [90]:
# afl = afl[111]
predict_traj = prediction(afl.agent_traj,avg_points=5)

In [91]:
import matplotlib.pyplot as plt

import numpy as np


In [95]:
def get_multiple_forecasts(afl):
    predict_traj = []
    for i in range(9,3,-1):
        predict_traj.append(prediction(afl.agent_traj,avg_points=(i)))
    return np.array(predict_traj)

predict_traj = get_multiple_forecasts(afl)


In [97]:
output_all2 = {}
counter = 1
for data in afl:
    print('\r'+str(counter)+'/'+str(len(afl)),end="")
#     predict_traj = get_multiple_forecasts(data)
    seq_id = int(data.current_seq.name[:-4])
    output_all2[seq_id] = 2
    counter += 1
    


3570/78143

7570/78143

11264/78143

15232/78143

19568/78143

...

67562/78143

71288/78143

75061/78143

78060/78143

78143/78143

In [94]:
output_all2[seq_id].shape

(6, 30, 2)